# MGT-448 Assignment 1

In [1]:
import torch
import numpy as np

dtype = torch.double
device = torch.device("cuda:0")
np.set_printoptions(precision=3)

tolerance = 1e-5
def rel_error(x, y):
    return torch.max(torch.abs(x - y) / (torch.maximum(1e-8, torch.abs(x) + torch.abs(y))))

def sigmoid(x):
    return 1/(1+torch.exp(-x))

iteration = 20
reg = 1e-4

In [2]:
# load data
import pandas as pd
data_train = np.loadtxt(open('./logisticRegressionData/training.csv', "r"), delimiter=",", skiprows=0)
data_train = torch.from_numpy(data_train.astype(np.float64)).to(device)
print(data_train.size())
data_test = np.loadtxt(open('./logisticRegressionData/testing.csv', "r"), delimiter=",", skiprows=0)
data_test = torch.from_numpy(data_test.astype(np.float64)).to(device)
print(data_test.size())

torch.Size([198, 28])
torch.Size([325, 28])


In [3]:
x_train = data_train[:, 1:]
y_train = data_train[:, 0].view(data_train.size(0), 1)
x_test = data_test[:, 1:]
y_test = data_test[:, 0].view(data_test.size(0), 1)

## Binomial case

### 1
(a) In logistic regression, the classifier maps the vector-form input to classes. In binomial LR, if we represent the classes with binary numbers 0 or 1, and denote the ith input data as $x_i$ and output class as $y_i$, one typical example is $y_i = \text{sigmoid} (\theta^T x_i)$ where $x_i\in\mathbb{R}^d, \theta\in\mathbb{R}^{d}, y_i\in\mathbb{R}$. 

(b) Write $h_{\theta}(x_i) = \text{sigmoid}(\theta x_i)$. The likelihood function is 
$$L(\theta) = \Pi_{i=1}^n(h_{\theta}(x_i))^{y_i}(1-h_{\theta}(x_i))^{1-y_i}$$ 
We mainly use the log-likelihood function, which is 
$$l(\theta) = \Sigma_{i=1}^n y_i\ln(h_{\theta}(x_i))+(1-y_i)\ln(1-h_{\theta}(x_i))$$ 
Suppose we have $n$ input and output pairs. Combine all inputs to a single matrix $X$, each row of which is one input vector, then $X\in\mathbb{R}^{n\times d}$. Combine all output data as vector $y$. Then the mapping becomes $y = \text{sigmoid} (X \theta)$. Then the vectorized log-likelihood function is 
$$l(\theta) = y^T\ln(h_{\theta}(X))+(1-y)^T\ln(1-h_{\theta}(X))$$
The cost function would be $J(\theta) = - \frac{1}{n} l(\theta) + \frac{1}{2}\lambda\left\|\theta\right\|^2$. For the case of without regularization, the gradient of the loss function is 
$$\frac{\partial J(\theta)}{\partial \theta} = \frac{1}{n}X^T (h_{\theta}(X)-y)$$ and its hessian is 
$$H(\theta) = \frac{1}{n}X^T \text{diag} \left\{(1-h_{\theta}(X))h_{\theta}(X)\right\}X$$
By Newton's Method, the iteration for $\theta$ is 
$$\theta = \theta - (H(\theta))^{-1}\frac{\partial J(\theta)}{\partial \theta}$$
Our algorithm starts with initializing $\theta$ as zero vector and stops when $J(\theta)$ is smaller than a predifined value.

(c) Classification error is given by $1-\frac{1}{n} (y \text{ AND } (\text{sigmoid}(\theta X)>0.5))$.

(d)  1) the dependent variables should be binary. 2) the observations should be mutually independent and the inputs/independent variable are not highly correlated. 3) the dependent and independent variables are linearly correlated. 4) relatively large sample size.

## 2
In the update of $\theta$ in Newton's Method, we use 
$$\theta = \theta - (H(\theta))^{-1}\frac{\partial J(\theta)}{\partial \theta} + \lambda \theta$$

In [4]:
N, D_in, D_out = x_train.size(0), x_train.size(1), 1

y_train_binary = torch.fmod(y_train, 2)
# print(y_train_binary)
# theta_2 = torch.randn(1, D_out, device=device, dtype=dtype)

def log_likelyhood_bin_cal(hx, y):
    return torch.t(y).mm(torch.log(hx)) + torch.t(1-y).mm(torch.log(1-hx))

### Newtons's Method

In [5]:
i, loss, step_size = 0, 1.0, 1.0
theta_1 = torch.zeros(D_in, D_out, device=device, dtype=dtype)
while i < iteration and loss > tolerance:
    i = i+1
    # forward pass
    scores = x_train.mm(theta_1)
    # computing loss
    h_1 = sigmoid(scores)
    h_0 = 1 - h_1
    log_likelyhood = log_likelyhood_bin_cal(h_1, y_train_binary)
    loss = -log_likelyhood/N + 0.5*reg*torch.sum(theta_1*theta_1)
    print("Loss is: ", loss.item())
    
    # optimization
    grad_loss = torch.t(x_train).mm(h_1-y_train_binary)
    # print(grad_loss.size())
    temp_diag = torch.diag(torch.flatten(h_0*h_1 + tolerance))
    # print(temp_diag.size(), x_train.size())
    hessian_loss = torch.t(x_train).mm(temp_diag).mm(x_train)
    dtheta_1 = torch.inverse(hessian_loss).mm(grad_loss) 
    theta_1 -= step_size * dtheta_1 + reg*theta_1
    
print("\nParameters theta_1 is:\n", theta_1.data.cpu().numpy())

Loss is:  0.6931471805599452
Loss is:  0.24310494300010352
Loss is:  0.11022913121958734
Loss is:  0.051880942051840746
Loss is:  0.02438109485788122
Loss is:  0.011814006920286325
Loss is:  0.007395360643426095
Loss is:  0.007213385523406105
Loss is:  nan

Parameters theta_1 is:
 [[ 1.057e+00]
 [ 2.273e+00]
 [-2.084e+00]
 [ 3.072e-03]
 [-7.010e+00]
 [ 6.582e+00]
 [-7.115e-01]
 [ 9.534e-01]
 [-3.766e+00]
 [ 1.439e+00]
 [ 1.329e+00]
 [-1.598e+00]
 [ 4.141e-01]
 [-6.146e+00]
 [ 6.085e+00]
 [-6.501e-01]
 [ 6.027e-01]
 [-1.540e+00]
 [ 1.144e+00]
 [ 2.042e-01]
 [-6.010e-01]
 [ 1.599e-01]
 [-3.249e+00]
 [ 3.820e+00]
 [-1.089e+00]
 [-6.903e-01]
 [ 6.860e-01]]


In [6]:
# check accuracy for test data
y_test_binary = torch.fmod(y_test, 2)
y_test_cal = x_test.mm(theta_1)
sigmoid_y = 1/(1+torch.exp(-y_test_cal)) > 0.5
# print(sigmoid_y)
accuracy = torch.true_divide(torch.sum(torch.abs(y_test_binary * sigmoid_y.int())), y_test_binary.size(0))
print("Accuracy is ", accuracy.item())

Accuracy is  0.6861538461538461


### Backtracking line search for Newton's Method in binomial case

In [7]:
i, loss, step_size = 0, 1.0, 1.0
theta_1 = torch.zeros(D_in, D_out, device=device, dtype=dtype)

alpha, rho, c = 1.0, 0.5, 0.5
criterion_v = torch.ones(1, 1, device=device, dtype=dtype)

while criterion_v and loss > tolerance and i < iteration:
    i = i+1
    # forward pass
    scores = x_train.mm(theta_1)
    # computing loss
    h_1 = sigmoid(scores)
    h_0 = 1 - h_1
    log_likelyhood = log_likelyhood_bin_cal(h_1, y_train_binary)
    loss = -log_likelyhood/N + 0.5*reg*torch.sum(theta_1*theta_1)
    print("Loss is: ", loss.item())
    
    # optimization
    grad_loss = torch.t(x_train).mm(h_1-y_train_binary)
    # print(grad_loss.size())
    temp_diag = torch.diag(torch.flatten(h_0*h_1 + tolerance))
    # print(temp_diag.size(), x_train.size())
    hessian_loss = torch.t(x_train).mm(temp_diag).mm(x_train)
    dtheta_1 = torch.inverse(hessian_loss).mm(grad_loss) 
    theta_1 -= step_size * dtheta_1 + reg*theta_1
    
    # backtracking linear search related
    pk = -dtheta_1
    alpha = rho*alpha
    criterion = (-log_likelyhood_bin_cal(sigmoid(x_train.mm(theta_1+alpha*pk)), y_train_binary)/N \
                + log_likelyhood_bin_cal(h_1, y_train_binary)/N \
                - c*alpha*torch.t(grad_loss/N).mm(pk)) > 0
    criterion_v = torch.sum(criterion.int())
    print("Criterion function value is: ", criterion)
    
print("\nParameters theta_1 is:\n", theta_1.data.cpu().numpy())

Loss is:  0.6931471805599452
Criterion function value is:  tensor([[False]], device='cuda:0')

Parameters theta_1 is:
 [[-0.122]
 [ 0.066]
 [ 0.03 ]
 [-0.022]
 [-0.517]
 [ 0.382]
 [ 0.176]
 [ 0.436]
 [-0.433]
 [-0.043]
 [-0.045]
 [ 0.098]
 [-0.018]
 [-0.486]
 [ 0.445]
 [ 0.176]
 [ 0.33 ]
 [-0.267]
 [-0.029]
 [-0.013]
 [ 0.108]
 [ 0.008]
 [-0.189]
 [ 0.185]
 [ 0.063]
 [-0.332]
 [ 0.302]]


In [8]:
# check accuracy for test data
y_test_binary = torch.fmod(y_test, 2)
y_test_cal = x_test.mm(theta_1)
sigmoid_y = 1/(1+torch.exp(-y_test_cal)) > 0.5
# print(sigmoid_y)
accuracy = torch.true_divide(torch.sum(torch.abs(y_test_binary * sigmoid_y.int())), y_test_binary.size(0))
print("Accuracy is ", accuracy.item())

Accuracy is  0.676923076923077


### Gradient descent

In [9]:
i, loss, step_size = 0, 1.0, 1.0
decay_rate = 0.5
theta_1 = torch.zeros(D_in, D_out, device=device, dtype=dtype)
while i < iteration and (loss > tolerance or torch.isinf(loss)):
    i = i+1
    step_size = step_size * decay_rate
    # forward pass
    scores = x_train.mm(theta_1)
    # computing loss
    h_1 =sigmoid(scores)
    h_0 = 1 - h_1
    log_likelyhood = log_likelyhood_bin_cal(h_1, y_train_binary)
    loss = -log_likelyhood/N + 0.5*reg*torch.sum(theta_1*theta_1)
    print("Loss is: ", loss.item())
    
    # optimization
    grad_loss = torch.t(x_train).mm(h_1-y_train_binary)
    dtheta_1 = grad_loss/N 
    theta_1 -= step_size * dtheta_1 + reg*theta_1
    
print("\nParameters theta_1 is:\n", theta_1.data.cpu().numpy())

Loss is:  0.6931471805599452
Loss is:  nan

Parameters theta_1 is:
 [[-7.615]
 [-4.415]
 [-6.437]
 [-9.837]
 [-5.34 ]
 [-8.709]
 [-8.001]
 [-3.585]
 [-6.566]
 [-0.545]
 [ 0.781]
 [-0.443]
 [ 2.914]
 [ 3.531]
 [ 4.412]
 [ 2.539]
 [ 1.466]
 [ 2.585]
 [ 1.387]
 [ 0.037]
 [ 0.274]
 [ 1.466]
 [ 0.062]
 [ 0.302]
 [ 1.242]
 [ 0.054]
 [ 0.25 ]]


In [10]:
# check accuracy for test data
y_test_binary = torch.fmod(y_test, 2)
y_test_cal = x_test.mm(theta_1)
sigmoid_y = 1/(1+torch.exp(-y_test_cal)) > 0.5
# print(sigmoid_y)
accuracy = torch.true_divide(torch.sum(torch.abs(y_test_binary * sigmoid_y.int())), y_test_binary.size(0))
print("Accuracy is ", accuracy.item())

Accuracy is  0.0


### Stochastic gradient descent

In [11]:
step_size, decay_rate = 1.0, 0.5

batch_sizes = [1, 16, 32, N]
theta_s = torch.zeros(D_in, D_out, len(batch_sizes), device=device, dtype=dtype)

y_test_binary = torch.fmod(y_test, 2)

for k in range(len(batch_sizes)):
    batch_size = batch_sizes[k]
    loss = 1.0
    theta_1 = torch.zeros(D_in, D_out, device=device, dtype=dtype)
    perm_seq = torch.randperm(N)
    x_train_perm, y_train_binary_perm = x_train[perm_seq, :], y_train_binary[perm_seq, :]
    
    for j in range(int(np.ceil(N/batch_size))):
        # print("j is ", j)
        start_ind = batch_size*j
        end_ind = np.minimum(N, start_ind+batch_size)
        # print("end_ind is ",  end_ind)
        x_batch = x_train_perm[start_ind:end_ind]
        y_batch = y_train_binary_perm[start_ind:end_ind]
        N_batch = end_ind - start_ind
        
        i = 0
        while i < iteration and (loss > tolerance or torch.isinf(loss)):
            i = i+1
            step_size = step_size * decay_rate
            # forward pass
            scores = x_batch.mm(theta_1)
            # computing loss
            h_1 = sigmoid(scores)
            h_0 = 1 - h_1
            log_likelyhood = log_likelyhood_bin_cal(h_1, y_batch)
            loss = -log_likelyhood/N_batch + 0.5*reg*torch.sum(theta_1*theta_1)
            print("Loss for batch size of ", batch_size, " is: ", loss.item())
            # optimization
            grad_loss = torch.t(x_batch).mm(h_1-y_batch)
            dtheta_1 = grad_loss/N_batch 
            theta_1 -= step_size * dtheta_1 + reg*theta_1
    
    theta_s[:, :, k] = theta_1
    # print("\nParameters theta_1 is:\n", theta_1.data.cpu().numpy())
    y_test_cal = x_test.mm(theta_1)
    sigmoid_y = sigmoid(y_test_cal) > 0.5
    # print(sigmoid_y)
    accuracy = torch.true_divide(torch.sum(torch.abs(y_test_binary * sigmoid_y.int())), y_test_binary.size(0))
    print("Accuracy for batch size of ", batch_size, " is ", accuracy.item(), "\n")


Loss for batch size of  1  is:  0.6931471805599453
Loss for batch size of  1  is:  nan
Accuracy for batch size of  1  is  0.0 

Loss for batch size of  16  is:  0.6931471805599453
Loss for batch size of  16  is:  63.746611187724454
Loss for batch size of  16  is:  nan
Accuracy for batch size of  16  is  0.7415384615384616 

Loss for batch size of  32  is:  0.6931471805599453
Loss for batch size of  32  is:  nan
Accuracy for batch size of  32  is  0.0 

Loss for batch size of  198  is:  0.6931471805599451
Loss for batch size of  198  is:  3.989342096473571
Loss for batch size of  198  is:  20.609948201080222
Loss for batch size of  198  is:  4.775562067711667
Loss for batch size of  198  is:  1.5170529384595168
Loss for batch size of  198  is:  0.4736798617308025
Loss for batch size of  198  is:  0.3577950947049763
Loss for batch size of  198  is:  0.3565033691348357
Loss for batch size of  198  is:  0.35604808863576726
Loss for batch size of  198  is:  0.3558405515645653
Loss for batch

## Multinomial case

In [12]:
num_classes = 4
N, D_in, D_out = x_train.size(0), x_train.size(1), num_classes

y_train_expand = torch.zeros(y_train.size(0), num_classes, device=device, dtype=dtype)
# print(y_train_expand.size())
for i in range(num_classes):
    temp = y_train == i+1
    y_train_expand[:, i] = torch.flatten(temp.int())

def log_likelyhood_mul_cal(hx, y):
    temp = torch.t(y).mm(torch.log(hx)) + torch.t(1-y).mm(torch.log(1-hx))
    return torch.diag(temp).view(num_classes, 1)
    

### Newton's Method

In [13]:
i, step_size = 0, 1.0
loss = torch.ones(num_classes, device=device, dtype=dtype)
theta_2 = torch.zeros(D_in, D_out, device=device, dtype=dtype)

while i < iteration and torch.max(loss) > tolerance:
    i = i+1
    # forward pass
    scores = x_train.mm(theta_2)
    # computing loss
    h_1 = 1 / (1+torch.exp(-scores))
    h_0 = 1 - h_1
    log_likelyhood = log_likelyhood_mul_cal(h_1, y_train_expand)
    loss = -log_likelyhood/N + 0.5*reg*torch.diag(torch.t(theta_2).mm(theta_2)).view(num_classes, 1)
    print("Loss is ", loss.data.cpu().numpy().T)
    
    # optimization
    grad_loss = torch.t(x_train).mm(h_1-y_train_expand)
    hessian_loss = torch.zeros(D_in, D_in, num_classes, device=device, dtype=dtype)
    dtheta_2 = torch.zeros(theta_2.size(), device=device, dtype=dtype)
    for i in range(num_classes):
        temp_diag = torch.diag(torch.flatten(h_0[:, i]*h_1[:, i]+tolerance))
        hessian_loss[:, :, i] = torch.t(x_train).mm(temp_diag).mm(x_train)
        temp = torch.inverse(hessian_loss[:, :, i]).mm(grad_loss[:, i].view(D_in, 1))
        dtheta_2[:, i] = torch.flatten(torch.inverse(hessian_loss[:, :, i]).mm(grad_loss[:, i].view(D_in, 1)))
    theta_2 -= step_size * dtheta_2 + reg*theta_2    
    
print("\nParameters theta_2 is:\n", theta_2.data.cpu().numpy())

Loss is  [[0.693 0.693 0.693 0.693]]
Loss is  [[0.286 0.201 0.216 0.179]]
Loss is  [[0.175 0.09  0.108 0.08 ]]
Loss is  [[0.106 0.043 0.057 0.039]]
Loss is  [[0.066 0.02  0.029 0.02 ]]
Loss is  [[0.034 0.01  0.015 0.01 ]]
Loss is  [[0.015 0.006 0.009 0.007]]
Loss is  [[0.007 0.005 0.009   nan]]

Parameters theta_2 is:
 [[ 1.147 -0.278 -0.757 -0.69 ]
 [-1.96  -2.543 -1.328 -0.947]
 [-1.082  1.433  0.903  1.148]
 [-0.116 -0.197 -0.243 -0.778]
 [-0.708  6.223  0.516 -7.925]
 [ 0.803 -2.69   3.032  1.651]
 [-0.672 -0.155  0.836  1.806]
 [-0.064 -1.635  7.541  1.308]
 [ 1.176  2.103 -7.393 -0.224]
 [ 0.639 -0.745 -0.016 -0.417]
 [-1.648 -1.006 -1.494 -0.561]
 [ 0.364  0.268  0.598  0.818]
 [ 0.601 -0.462 -0.251 -0.971]
 [-1.464  5.625  0.135 -8.616]
 [ 0.789 -2.332  3.184  1.99 ]
 [-0.42  -0.389  0.519  1.818]
 [ 0.213 -0.98   6.359  1.529]
 [ 1.388  1.982 -4.876 -1.632]
 [ 0.31  -0.263 -0.517  0.505]
 [-1.147 -0.117 -0.397  2.351]
 [-0.081 -0.032  2.351 -2.2  ]
 [ 0.213 -0.219 -0.217 -1.29

In [14]:
# check accuracy for test data
y_test_expand = torch.zeros(y_test.size(0), num_classes, device=device, dtype=dtype)
# print(y_test_expand.size())
for i in range(num_classes):
    temp = y_test == i+1
    y_test_expand[:, i] = torch.flatten(temp.int())

y_test_cal = x_test.mm(theta_2)
sigmoid_y = 1/(1+torch.exp(-y_test_cal)) > 0.5
# print(sigmoid_y)
accuracy = torch.true_divide(torch.sum(torch.abs(y_test_expand * sigmoid_y.int())), y_test_expand.size(0))
print("Accuracy is ", accuracy.item())

Accuracy is  0.7876923076923077


### Backtrack line search for Newton's Method in multinomial case

In [15]:
i, step_size = 0, 1.0
loss = torch.ones(num_classes, device=device, dtype=dtype)
theta_2 = torch.zeros(D_in, D_out, device=device, dtype=dtype)

alpha, rho, c = 1.0, 0.5, 0.5
criterion_v = torch.ones(1, 1, device=device, dtype=dtype)

while i < iteration and torch.max(loss) > tolerance and criterion_v:
    i = i+1
    # forward pass
    scores = x_train.mm(theta_2)
    # computing loss
    h_1 = sigmoid(scores)
    h_0 = 1 - h_1
    log_likelyhood = log_likelyhood_mul_cal(h_1, y_train_expand)
    loss = -log_likelyhood/N + 0.5*reg*torch.diag(torch.t(theta_2).mm(theta_2)).view(num_classes, 1)
    print("Loss is ", loss.data.cpu().numpy().T)
    
    # optimization
    grad_loss = torch.t(x_train).mm(h_1-y_train_expand)
    hessian_loss = torch.zeros(D_in, D_in, num_classes, device=device, dtype=dtype)
    dtheta_2 = torch.zeros(theta_2.size(), device=device, dtype=dtype)
    for i in range(num_classes):
        temp_diag = torch.diag(torch.flatten(h_0[:, i]*h_1[:, i]+tolerance))
        hessian_loss[:, :, i] = torch.t(x_train).mm(temp_diag).mm(x_train)
        temp = torch.inverse(hessian_loss[:, :, i]).mm(grad_loss[:, i].view(D_in, 1))
        dtheta_2[:, i] = torch.flatten(torch.inverse(hessian_loss[:, :, i]).mm(grad_loss[:, i].view(D_in, 1)))
    theta_2 -= step_size * dtheta_2 + reg*theta_2
    
    # backtracing line search related
    pk = -dtheta_2
    alpha = rho*alpha
    criterion = (-log_likelyhood_mul_cal(x_train.mm(theta_2+alpha*pk), y_train_expand)/N \
                +log_likelyhood_mul_cal(h_1, y_train_expand)/N \
                -c*alpha*torch.diag(torch.t(grad_loss/N).mm(pk)).view(num_classes, 1))>0
    criterion_v = torch.sum(criterion.int())
    print("Criterion function value is: ", criterion_v.item())
    
print("\nParameters theta_2 is:\n", theta_2.data.cpu().numpy())

Loss is  [[0.693 0.693 0.693 0.693]]
Criterion function value is:  0

Parameters theta_2 is:
 [[-0.055  0.114 -0.073  0.002]
 [-0.029 -0.027  0.096 -0.038]
 [ 0.13  -0.071 -0.101  0.04 ]
 [ 0.101 -0.041 -0.125  0.061]
 [ 0.266  0.357 -0.824  0.121]
 [-0.249 -0.092  0.658 -0.263]
 [-0.012 -0.115  0.194 -0.053]
 [ 0.057 -0.289  0.441 -0.086]
 [-0.069  0.236 -0.439  0.122]
 [-0.043  0.025 -0.006  0.012]
 [-0.169  0.168  0.125 -0.123]
 [ 0.251 -0.201 -0.154  0.102]
 [ 0.093 -0.028 -0.113  0.044]
 [ 0.261  0.365 -0.786  0.081]
 [-0.119 -0.172  0.589 -0.247]
 [ 0.073 -0.148  0.111 -0.021]
 [ 0.192 -0.278  0.2    0.01 ]
 [-0.218  0.253 -0.125 -0.062]
 [-0.047  0.012  0.016  0.016]
 [-0.032  0.035  0.027 -0.015]
 [ 0.075 -0.071  0.027 -0.043]
 [ 0.052 -0.016 -0.046  0.007]
 [ 0.038  0.264 -0.238 -0.085]
 [ 0.067 -0.193  0.128  0.019]
 [ 0.017 -0.051  0.049 -0.01 ]
 [-0.433  0.219  0.114  0.125]
 [ 0.398 -0.21  -0.118 -0.115]]


In [16]:
# check accuracy for test data
y_test_expand = torch.zeros(y_test.size(0), num_classes, device=device, dtype=dtype)
# print(y_test_expand.size())
for i in range(num_classes):
    temp = y_test == i+1
    y_test_expand[:, i] = torch.flatten(temp.int())

y_test_cal = x_test.mm(theta_2)
sigmoid_y = 1/(1+torch.exp(-y_test_cal)) > 0.5
# print(sigmoid_y)
accuracy = torch.true_divide(torch.sum(torch.abs(y_test_expand * sigmoid_y.int())), y_test_expand.size(0))
print("Accuracy is ", accuracy.item())

Accuracy is  0.7661538461538462


### Gradient descent

In [17]:
i, step_size = 0, 1.0
decay_rate = 0.8
loss = torch.ones(num_classes, device=device, dtype=dtype)
theta_2 = torch.zeros(D_in, D_out, device=device, dtype=dtype)
while i < iteration and (torch.max(loss) > tolerance or torch.sum(torch.isinf(loss).int())):
    i = i+1
    step_size = step_size * decay_rate
    # forward pass
    scores = x_train.mm(theta_2)
    # computing loss
    h_1 = sigmoid(scores)
    h_0 = 1 - h_1
    log_likelyhood = log_likelyhood_mul_cal(h_1, y_train_expand)
    loss = -log_likelyhood/N + 0.5*reg*torch.diag(torch.t(theta_2).mm(theta_2)).view(num_classes, 1)
    print("Loss is ", loss.data.cpu().numpy().T)
    
    # optimization
    grad_loss = torch.t(x_train).mm(h_1-y_train_expand)
    theta_2 -= step_size * grad_loss/N + reg*theta_2
    
print("\nParameters theta_2 is:\n", theta_2.data.cpu().numpy())

Loss is  [[0.693 0.693 0.693 0.693]]
Loss is  [[nan nan nan nan]]

Parameters theta_2 is:
 [[-3.059e-01  2.145e+00 -4.240e+00 -7.666e+00]
 [-4.480e+00 -6.145e+00  2.567e+00  1.498e+00]
 [-3.564e+00 -6.218e+00  1.396e+00 -1.797e+00]
 [-4.527e-01 -8.818e-01 -2.205e+00 -1.268e+01]
 [-2.175e+00 -5.957e+00  1.323e+00 -2.583e+00]
 [-9.279e-01 -7.008e+00  3.157e-01 -8.477e+00]
 [-3.402e+00 -2.028e+00  2.539e+00 -1.160e+01]
 [-1.054e+00 -2.712e+00 -1.236e+00  4.146e-01]
 [-5.544e-01 -3.478e+00 -2.271e+00 -3.471e+00]
 [ 3.551e+00 -8.046e+00  3.536e+00 -7.168e+00]
 [ 5.922e+00  3.443e+00 -2.853e+00 -8.080e+00]
 [ 6.274e+00  1.035e+00 -1.918e+00 -1.060e+01]
 [ 3.136e+00 -4.012e+00  1.835e+00 -3.359e-01]
 [ 2.889e+00  4.632e+00 -1.459e+00 -7.173e-01]
 [ 2.602e+00  3.830e+00 -6.033e-01  6.397e-01]
 [ 5.593e+00 -1.856e+00 -2.944e+00  1.431e+00]
 [ 1.896e+00  1.217e+00  1.079e+00 -4.353e+00]
 [ 2.104e+00  6.456e-01  1.964e+00 -3.819e+00]
 [-6.528e-01  1.066e+00  1.099e-01  2.681e+00]
 [-4.626e-02  1.

In [18]:
# check accuracy for test data
y_test_expand = torch.zeros(y_test.size(0), num_classes, device=device, dtype=dtype)
# print(y_test_expand.size())
for i in range(num_classes):
    temp = y_test == i+1
    y_test_expand[:, i] = torch.flatten(temp.int())

y_test_cal = x_test.mm(theta_2)
sigmoid_y = 1/(1+torch.exp(-y_test_cal)) > 0.5
# print(sigmoid_y)
accuracy = torch.true_divide(torch.sum(torch.abs(y_test_expand * sigmoid_y.int())), y_test_expand.size(0))
print("Accuracy is ", accuracy.item())

Accuracy is  0.3323076923076923


### 5

The gradient descent has less computation when updating the parameters, while extra parameter, like the decay rate should be set as it's likely that the results from gradient descent is ocsillating arount the optimal point and the step size should decrease during the iterations. 

### Stochastic gradient descent

In [19]:
step_size, decay_rate = 1.0, 0.5

batch_sizes = [1, 16, 32, N]
theta_s = torch.zeros(D_in, D_out, len(batch_sizes), device=device, dtype=dtype)

y_test_expand = torch.zeros(y_test.size(0), num_classes, device=device, dtype=dtype)
# print(y_test_expand.size())
for i in range(num_classes):
    temp = y_test == i+1
    y_test_expand[:, i] = torch.flatten(temp.int())

for k in range(len(batch_sizes)):
    batch_size = batch_sizes[k]
    loss = torch.ones(num_classes, device=device, dtype=dtype)
    theta_2 = torch.zeros(D_in, D_out, device=device, dtype=dtype)
    perm_seq = torch.randperm(N)
    x_train_perm, y_train_expand_perm = x_train[perm_seq, :], y_train_expand[perm_seq, :]
    
    for j in range(int(np.ceil(N/batch_size))):
        # print("j is ", j)
        start_ind = batch_size*j
        end_ind = np.minimum(N, start_ind+batch_size)
        # print("end_ind is ",  end_ind)
        x_batch = x_train_perm[start_ind:end_ind]
        y_batch = y_train_expand_perm[start_ind:end_ind]
        N_batch = end_ind - start_ind
        
        i = 0
        while i < iteration and (torch.max(loss) > tolerance or torch.sum(torch.isinf(loss).int())):
            i = i+1
            step_size = step_size * decay_rate
            # forward pass
            scores = x_batch.mm(theta_2)
            # computing loss
            h_1 = sigmoid(scores)
            h_0 = 1 - h_1
            log_likelyhood = log_likelyhood_mul_cal(h_1, y_batch)
            loss = -log_likelyhood/N + 0.5*reg*torch.diag(torch.t(theta_2).mm(theta_2)).view(num_classes, 1)
            print("Loss for batch size of ", batch_size, " is: ", loss.data.cpu().numpy().T)
            # optimization
            grad_loss = torch.t(x_batch).mm(h_1-y_batch)
            theta_2 -= step_size * grad_loss/N + reg*theta_2
    
    theta_s[:, :, k] = theta_2
    # print("\nParameters theta_1 is:\n", theta_1.data.cpu().numpy())
    y_test_cal = x_test.mm(theta_2)
    sigmoid_y = sigmoid(y_test_cal) > 0.5
    # print(sigmoid_y)
    accuracy = torch.true_divide(torch.sum(torch.abs(y_test_binary * sigmoid_y.int())), y_test_binary.size(0))
    print("Accuracy for batch size of ", batch_size, " is ", accuracy.item(), "\n")


Loss for batch size of  1  is:  [[0.004 0.004 0.004 0.004]]
Loss for batch size of  1  is:  [[4.942e-06       nan 4.942e-06 4.942e-06]]
Accuracy for batch size of  1  is  0.7415384615384616 

Loss for batch size of  16  is:  [[0.056 0.056 0.056 0.056]]
Loss for batch size of  16  is:  [[2.369 2.556 1.278 2.487]]
Loss for batch size of  16  is:  [[0.004 0.398 1.178 0.412]]
Loss for batch size of  16  is:  [[0.004 1.035 1.128 0.276]]
Loss for batch size of  16  is:  [[0.004 0.498 1.103 0.421]]
Loss for batch size of  16  is:  [[0.003 0.228 1.09  0.162]]
Loss for batch size of  16  is:  [[0.003 0.094 1.084 0.06 ]]
Loss for batch size of  16  is:  [[0.003 0.037 1.081 0.042]]
Loss for batch size of  16  is:  [[0.003 0.022 1.079 0.034]]
Loss for batch size of  16  is:  [[0.003 0.018 1.078 0.031]]
Loss for batch size of  16  is:  [[0.003 0.017 1.078 0.029]]
Loss for batch size of  16  is:  [[0.003 0.016 1.078 0.029]]
Loss for batch size of  16  is:  [[0.003 0.016 1.077 0.028]]
Loss for batch 